# Voting Analysis

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import os
from datetime import datetime

In [2]:
cwd = os.getcwd()
date_format = "%Y-%m-%d"
TORNADO_PATH = os.path.join(cwd, "..", "data/tornadoes/US_tornadoes_2000_2022.csv")
TORNADO_SHAPEFILE = os.path.join(cwd, "..", "data/geographic/US_tornadoes_1950_2022/US_tornadoes_1950_2022.shp")
COUNTIES_PATH = os.path.join(cwd, "..", "data/counties/2018_counties.csv")
COUNTIES_SHAPEFILE = os.path.join(cwd, "..", "data/geographic/cb_2018_us_county_5m/cb_2018_us_county_5m.shp")
INTERSECTION_PATH = os.path.join(cwd, "..", "data/tornado_county_intersections.csv")
RESULTS_PATH = os.path.join(cwd, "..", "data/results_2000s.csv")

# Naive Comparison

In [3]:
def getCountyVoteSet_FlatCount(data_vote,data_tornado,year):
    output = np.array([[0.0,0]]* len(data_vote))
    
    i = 0
    for row in data_vote.index:
        
        filter1 = data_tornado.loc[data_tornado["GEOID"] == data_vote["fips"][row]]
        filter2 = filter1.loc[filter1["yr"].isin(range(year-4,year+1))]
        output[i,1] = len(filter2)
        output[i,0] = data_vote[str(year)][row]
        # print(i)
        i+=1
    
    return output

def getCountyVoteSet_CostAgg(data_vote,data_tornado,year):
    output = np.array([[0.0,0]]* len(data_vote))
    
    i = 0
    for row in data_vote.index:
        
        filter1 = data_tornado.loc[data_tornado["GEOID"] == data_vote["fips"][row]]
        filter2 = filter1.loc[filter1["yr"].isin(range(year-4,year+1))]
        totalCost = sum(filter2["loss"])
        output[i,1] = totalCost
        output[i,0] = data_vote[str(year)][row]
        # print(i)
        i+=1
    
    return output

def getCountyVoteSet_Casualties(data_vote,data_tornado,year):
    output = np.array([[0.0,0]]* len(data_vote))
    
    i = 0
    for row in data_vote.index:
        
        filter1 = data_tornado.loc[data_tornado["GEOID"] == data_vote["fips"][row]]
        filter2 = filter1.loc[filter1["yr"].isin(range(year-4,year+1))]
        totalCost = sum(filter2["inj"]) + sum(filter2["fat"])
        output[i,1] = totalCost
        output[i,0] = data_vote[str(year)][row]
        # print(i)
        i+=1
    
    return output

    
    


In [12]:
data_tornado = pd.read_csv(INTERSECTION_PATH)
data_voting = pd.read_csv(RESULTS_PATH)

# for year in range(2004, 2021, 4):
#     dist_Dem = data_voting.loc[data_voting[str(year)].notnull()]
#     scatters = getCountyVoteSet_Casualties(dist_Dem,data_tornado,year)
#     print(year)
#     plt.scatter(scatters[:,1],scatters[:,0])
#     plt.title(f"{year} Disaster/Voter Turnout")
#     plt.ylabel("% Democrat Share")
#     plt.xlabel("Injuries and Fatalities")
#     plt.show()
    
    
# for year in range(2004, 2021, 4):
#     dist_Dem = data_voting.loc[data_voting[str(year)].notnull()]
    
#     scatters = getCountyVoteSet_CostAgg(dist_Dem,data_tornado,year)
#     print(year)
#     plt.scatter(scatters[:,1],scatters[:,0])
#     plt.title(f"{year} Tornado/Voter Turnout")
#     plt.ylabel("% Democrat Share")
#     plt.xlabel("Monetary Loss")
#     plt.show()
    
# for year in range(2004, 2021, 4):
#     dist_Dem = data_voting.loc[data_voting[str(year)].notnull()]
#     scatters = getCountyVoteSet_FlatCount(dist_Dem,data_tornado,year)
#     print(year)
#     plt.scatter(scatters[:,1],scatters[:,0])
#     plt.title(f"{year} Tornado/Voter Turnout")
#     plt.ylabel("% Democrat Share")
#     plt.xlabel("Total tornadoes")
#     plt.show()
    



# Deviation Analysis

In [22]:


def getCountyVoteSet_MinDeath(data_tornado):


    filter1 = data_tornado[data_tornado["fat"] > 20]
    print(filter1)

def getCountyVoteSet_Money(data_tornado):


    filter1 = data_tornado[data_tornado["loss"] > 10000000]
    print(filter1)


data_tornado = pd.read_csv(TORNADO_PATH)
data_voting = pd.read_csv(RESULTS_PATH)
# getCountyVoteSet_Money(data_tornado)


In [ ]:
def find_synth(data_vote, data_tornado, county, election_year):
    """
    Find a set of counties that can act as a synthetic control for a county, having similar stats, and being far enough away.
    Omit the year that is being observed
    """
    pass
    